### constant_estimator.ipynb
- Siep Dokter
- Emil Jousimaa
- Oleksandr Sosovskyy
- Mario Gabriele Carofano

> This file contains the estimation of the path loss exponent and the power received at a reference distance as described in Task 2.

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re

In [46]:
def read_dataframes(directory_path, files):
    """
    Reads the filepaths and forms the dataframes.

    Parameters: 
    directory_path (string): The first part of the directory path.
    files (list): A list that contains the names of the files.

    Returns:
    A list that contains all the dataframes.
    """

    dataframes = []
    for power_values in files:
            current_df = pd.read_csv(directory_path + power_values, sep=';')
            dataframes.append(current_df)

    return dataframes

# https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [47]:
def resize_dataframes(dataframes):
    """
    Resizes the dataframes so that they are all the same size as the smallest one.

    Parameters:
    dataframes (list): A list that contains all the dataframes.

    Returns:
    A list of resized dataframes.
    """

    smallest_df = min(dataframes, key=len)

    resized_dataframes = [df.iloc[:len(smallest_df)] for df in dataframes]

    return len(smallest_df), resized_dataframes

In [48]:
def calculate_x_matrix(matrix_row_size, distances, smallest_df_size):
    """
    Calculates the x_matrix (X=[1  10log(d)]) which has the size of Lx2, where L is the matrix row size.

    Parameters:
    matrix_row_size (int): The size of the matrices' rows.
    distances (list): A list of different distances used in the measurements.
    smallest_df_size (int): Size of the smallest dataframe.

    Returns:
    the y_matrix described in the article.

    """
    distance_values = []
    running_number = 0
    for i in range(0, matrix_row_size):
        distance_values.append(-10 * math.log10(distances[running_number]))
        if (i + 1) % smallest_df_size == 0:
            running_number += 1

    return np.column_stack((np.ones(len(distance_values)), distance_values))

# https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
# https://docs.python.org/3/library/math.html

In [49]:
def calculate_y_matrix(resized_dataframes):
    """
    Calculates the y_matrix (y=[PL(d)]) which has the size of Lx1, where L is the matrix row size.

    Parameters:
    resized_dataframes (list): A list of dataframes that all have the same size as the smallest dataset.

    Returns:
    the y_matrix described in the article.

    """
    
    power_values = []

    for df in resized_dataframes:
        power_values.extend(df["Rx Power [dBm]"].tolist())

    y_matrix = np.column_stack(power_values)
    return np.transpose(y_matrix)

# https://numpy.org/doc/stable/reference/generated/numpy.matrix.html

In [50]:
directory_path = "../dataset/Scenario 0/RPI/RSS_BLT_Dataset/" 
files = ["Distanza5.csv", "Distanza10.csv", "Distanza15.csv", "Distanza20.csv", "Distanza25.csv"]

# Creates a list that contains the different distances as integers.
distances = [int(re.search(r'\d+', s).group()) for s in files]

dataframes = read_dataframes(directory_path, files)

smallest_df_size, resized_dataframes = resize_dataframes(dataframes)

matrix_row_size = smallest_df_size * len(resized_dataframes)

x_matrix = calculate_x_matrix(matrix_row_size, distances, smallest_df_size)
y_matrix = calculate_y_matrix(resized_dataframes)


# Calculation of the aplha estimation.
left_term = np.linalg.inv(np.matmul(np.transpose(x_matrix), x_matrix))

alpha_estimate = np.matmul(left_term, np.transpose(x_matrix))
alpha_estimate = np.matmul(alpha_estimate, y_matrix)
alpha_estimate = np.transpose(alpha_estimate)

print("The estimated power received at a reference distance:", alpha_estimate[0][0])
print("")
print("The estimated path loss exponent:", alpha_estimate[0][1])

The estimated power received at a reference distance: -75.00268290617765

The estimated path loss exponent: 0.3001957817116097
